# Day 3 (Context Engineering: Sessions & Memory) 

> ⚠️ **Nota importante**  
>  A continuación se muestra un proceso secuencial inicial, a nivel de configuración, que es necesario realizar antes de empezar a trabajar con los Agentes. Se importan librerias de ADK necesarias con sus respectivos componentes, se crean las Helper Functions y las opciones de configuración de Retry.  


In [2]:
from typing import Any, Dict

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [3]:
# Define helper functions that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

## Fin de Configuración

> ⚠️ **Nota importante**  
> Implementando nuestro primer agente con persistencia (Session). Puede recordar y tener conversaciones constructivas. ADK ofrece diferentes tipos de *session*, pero en este caso empezaremos por la mas sencilla: *InMemomrySessionService*  

    

In [5]:
APP_NAME = "default"
USER_ID = "default"
SESSION = "default"

MODEL_NAME = "gemini-2.5-flash-lite"



In [6]:
#Creando el Agente LLM

root_agent=Agent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options = retry_config),
    name="text_chat_bot",
    description="Un chat-bot de texto",
)   



In [7]:
# Paso 2: Configurar la gestión de session
# InMemorySessionService almacena las conversaciones en la RAM (temporalmente)

session_service = InMemorySessionService()



In [8]:
#Creamos el Runner

runner=Runner(agent=root_agent, app_name=APP_NAME,session_service=session_service)

print("Agente con persistencia (memoria) iniciado")
print("")
print(f"Aplicacion : {APP_NAME}")
print(f"Usuario : {USER_ID}")
print(f"Usando : {session_service.__class__.__name__}")

Agente con persistencia (memoria) iniciado

Aplicacion : default
Usuario : default
Usando : InMemorySessionService


> ⚠️ **Nota importante**
> 
> Pruebas del Agente. Ahora veremos la magia de Sessions y la persistencia temporal:  


In [9]:
#Prueba de conversación con dos consultas en la misma Session
#las dos consultas son parte de la misma SESSION, por lo tanto se mantiene el CONTEXTO


await run_session(

    runner,

    [
        "Hola!, me llamo Héctor. ¿Cual es la localidad con menos población en cataluña?",

         "Hola, ¿Cual es mi nombre?"
    ],

    "statefull-agentic-session",

)




 ### Session: statefull-agentic-session

User > Hola!, me llamo Héctor. ¿Cual es la localidad con menos población en cataluña?
gemini-2.5-flash-lite >  ¡Hola, Héctor! La localidad con menos población en Cataluña es **Espluga de Serra**, en la comarca de la Alta Ribagorça.

Según los datos más recientes, su población es muy reducida, a menudo por debajo de las 10 personas. Es un lugar con un encanto rural muy especial.

User > Hola, ¿Cual es mi nombre?
gemini-2.5-flash-lite >  Tu nombre es Héctor.


In [13]:
#nMemorySessionService es temporal. Una vez que la aplicación se detiene, todo el historial de la conversación se pierde

> ⚠️ **Nota importante**
> 
> Para que haya persistencia real, tiene que serintegrada, haciendo uso de DDBB.
> 
> Ahora haremos uso de *DatabaseSessionService*

In [10]:
#Creamos el mismo agente chat-bot de antes (Advierte que para esta creación , usamos LlmAgent)

chatbot_agent=LlmAgent(

    model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
    name="text_chat_bot",
    description="Un Chat-bot de texto con persistencia"

)
    


In [11]:
#Cambiamos a DatabaseSessionService
#La base de datos SQlite será creada automaticamente

db_url="sqlite+aiosqlite:///my_agent_data.db"  #Archivo local de SQLite
session_service=DatabaseSessionService(db_url=db_url)


In [1]:
#Creamos un nuevo Runner con persistencia storage

runner=Runner(agent=chatbot_agent, app_name=APP_NAME, session_service=session_service)

print("Actualizado para persistencia con base de datos")
print("")
print(f"Nombre de la base de datos: {db_url}")
print("")
print("Ahora Sessions sobrevive a los crasheos, restarts etc")

NameError: name 'Runner' is not defined

> ⚠️ **Nota importante**
> 
> Vamos a proceder ahora a verificar la persistencia en DDBB. La ID de la SESSION será *test-db-session-01*

In [15]:
await run_session(
    runner,
    ["Hola, me llamo Héctor. Cual es la capital de España?","Como  me llamo?"],
    "test-db-session-01"
)




 ### Session: test-db-session-01

User > Hola, me llamo Héctor. Cual es la capital de España?
gemini-2.5-flash-lite >  Hola Héctor, la capital de España es Madrid.

User > Como  me llamo?
gemini-2.5-flash-lite >  Te llamas Héctor.


In [19]:
#Si cambiamos la SESSION, no podra acceder a los datos de la DDBB. En este caso usamos otra SESSION con ID "test-db-session-02"

await run_session(
    runner,
    ["Como me llamo?"],
    "test-db-session-02"
)



 ### Session: test-db-session-02

User > Como me llamo?
gemini-2.5-flash-lite >  Lo siento, pero como agente de IA, no tengo forma de saber tu nombre. No guardo información personal sobre los usuarios.


> ⚠️ **Nota importante**
> 
> Todo se va guardando en la BBDD, por lo tanto enseguida nos encontramos con que es enorme y empieza a bajar su rendimiento.

> ⚠️ **Nota importante**
> 
> Pero ¿y si pudiéramos resumir automáticamente el pasado? Usemos la función de Compactación de Contexto de ADK para ver cómo reducir automáticamente el contexto que se almacena en la sesión.

In [20]:
# Redefinimos nuestra app con el modo compactación

research_app_compacting = App(
    name="research_app_compacting",
    root_agent=chatbot_agent,
    
    # Esta parte es la nueva, la compactación
    events_compaction_config=EventsCompactionConfig(
        compaction_interval=3,  # Activar la compactación cada 3 invocaciones
        overlap_size=1,  # Mantener un truno previo como contexto
    ),
)

db_url = "sqlite+aiosqlite:///my_agent_data.db"  
session_service = DatabaseSessionService(db_url=db_url)

# Creamos un nuevo Runner para este Upgrade de APP
research_runner_compacting = Runner(
    app=research_app_compacting, session_service=session_service
)


print("✅Aplicacion actualizada con compactación de eventos en la Base de Datos")

✅Aplicacion actualizada con compactación de eventos en la Base de Datos


C:\Users\Trabajo\AppData\Local\Temp\ipykernel_10740\1307449722.py:8: UserWarning: [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


> ⚠️ **Nota importante**
> 
> Para todos los turnos futuros de esta conversación, el agente recibirá este **resumen** conciso en lugar del historial completo. Esto **reduce costos**, mejora el rendimiento y ayuda al agente a mantenerse enfocado en lo más importante.

> ⚠️ **Nota importante**
> 
> Por ultimo creamos un Agente con **CUSTOM TOOLS** para recuperar el **nombre de usuario** y la **nacionalidad** de una determinada **SESSION**

In [22]:
# Definir niveles de alcance para las claves de estado (siguiendo las mejores prácticas)

USER_NAME_SCOPE_LEVELS = ("temp", "user", "app")



In [26]:
# Esto demuestra cómo las herramientas pueden ESCRIBIR en el estado de la sesión usando tool_context.

# El prefijo 'user:' indica que se trata de datos específicos del usuario.

def save_user_info(ToolContext, username: str, country: str) -> Dict[str, Any]:

    """

    Herramienta para registrar y guardar el nombre de usuario y el país en el estado de la sesión.
    
        Argumentos:
        
        user_name: El nombre de usuario que se almacenará en el estado de la sesión
        
        country: El nombre del país del usuario

    """

    # Escribir en el estado de la sesión usando el prefijo 'user:' para datos del usuario

    tool_context.state["user:name"] = user_name
    tool_context.state["user:country"] = country

    return {"status": "success"}





# Esto es otra demostración de como TOOLS pueden lEER de session state

def retrieve_userinfo (tool_context:ToolContext) -> Dict[str,Any]:

    """

    Herramienta para recuperar el nombre de usuario y el país desde el estado de la sesión.


    """

    #Leer desde session state

    user_name = tool_context.state.get("user:name", "Username not found")
    country = tool_context.state.get("user:country", "Country not found")

    return {"status": "success", "user_name": user_name, "country": country}


print("Tools creadas con éxito")
     

Tools creadas con éxito


> ⚠️ **Nota importante**
> 
> Creando el Agente con las TOOLS recien codificadas arriba

In [28]:
# Configuración

APP_NAME="default"
USER_ID="default"
MODEL_NAME="gemini-2.5-flash-lite"

# Creación del Agente

root_agent = LlmAgent(
    
    model=Gemini(model=MODEL_NAME,rety_options=retry_config),

    name="text_chat_bot",

    description= """

                 Un chatbot de texto.
    
                    Herramientas para gestionar el contexto del usuario:
                    
                    Para registrar el nombre de usuario y el país cuando se proporcionen, usa la herramienta save_userinfo.
                    
                    Para obtener el nombre de usuario y el país cuando sea necesario, usa la herramienta retrieve_userinfo.


                 """,
    
    tools = [save_user_info, retrieve_userinfo],
)


# Iniciando SESSION SERVICE y RUNNER

session_service = InMemorySessionService()
runner = Runner(agent=root_agent, session_service=session_service, app_name="default")

print("Agente con TOOLS en SESSION inciado")



Agente con TOOLS en SESSION inciado


> ⚠️ **Nota importante**
>
> Probando como el agente usa SESSION STATE para recuperar la información a través de la conversación

In [34]:

await run_session(
    runner,
    [
        "Hola, que harás hoy?, Cual es mi nombre?",  # El agente aquí no debería conocer mi nombre
        "Me llamo Héctor, soy de España",  # Aqui se da el nombre y el país, el Agente debería recoger esta información
        "Como me llamo? De donde soy?",  # El Agente aqui debería recuper la información
    ],
    "state-demo-session",
)


 ### Session: state-demo-session

User > Hola, que harás hoy?, Cual es mi nombre?


_ResourceExhaustedError: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite\nPlease retry in 40.365659323s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_co

> ⚠️ **Nota importante**
>
> Inspección del estado de la sesión
>
> Inspeccionemos directamente el estado de la sesión para ver qué se ha almacenado:


In [33]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="state-demo-session"
)

print("SESSION STATE contiene:")
print(session.state)


SESSION STATE contiene:
{}


> ⚠️ **Nota importante**
>
> Por ultimo limpiamos la DATABASE (Opcional)

In [1]:
import os

if os.path.exists("my_agent_data.db"):
    os.remove("my_agent_data.db")
print("✅ Base de Datos vaciada")

✅ Base de Datos vaciada


---

# Day 3 (Context Engineering: Sessions & Memory) | PARTE 2

---

> ⚠️ **Nota importante**
>
> Segunda parte del manual del dia 3
>
> Aqui usaremos Memory, no Sessions. La diferencia radica en que Sessions es temporal, persistencia muy comparable a una memoria RAM, y en cambio Memory es mas como una persistencia a largo plazo, imitando el conocimiento.

Ejemplo: Imagina que hablas con un asistente personal:

🗣️ Session: Recuerda lo que dijiste hace 10 minutos en ESTA conversación


🧠 Memory: Recuerda tus preferencias de conversaciones de LA SEMANA PASADA